**Installation and imports**

In [ ]:
!pip install -U requests urllib3
!pip install -qU langchain-huggingface
!pip install langchain_milvus sentence-transformers langchain_community langchain_openai

In [ ]:
import os
from uuid import uuid4
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_milvus import Milvus
from langchain_core.documents import Document
from langchain_community.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

**Define queries**

In [ ]:
incident_id = os.getenv('incident_id')

In [ ]:
select_record_by_id_query = """
SELECT description, fracture_classification 
FROM fracture_incidents
WHERE id = %s;
"""

update_record_query = f"""
UPDATE fracture_incidents
SET 
    fracture_advice = %s
WHERE 
    id = %s;
"""

**Query database to get incident data**

In [ ]:
record = {}

try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor(cursor_factory=DictCursor)

    cursor.execute(select_record_by_id_query, (incident_id,))
    record = cursor.fetchone()
    
    if record:
        print("Retrieved record:")
        print(record)
    else:
        print(f"No record found with ID {incident_id}.")

except psycopg2.DatabaseError as e:
    print(f"Error: {e}")
finally:
    if conn:
        cursor.close()
        conn.close()
        print("Database connection closed.")

**Download embeddings**

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

**Setting up Milvus Lite Vector database**

In [ ]:
URI = "./medical_information.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
)

In [ ]:
vector_store_saved = Milvus.from_documents(
    [Document(page_content="foo!")],
    embeddings,
    collection_name="medical_information",
    connection_args={"uri": URI},
)

In [ ]:
vector_store_loaded = Milvus(
    embeddings,
    connection_args={"uri": URI},
    collection_name="patient_records",
)

**Extracting all information from documents to insert into vector database**

In [ ]:
def extract_text_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

docs_directory = "./docs/"
file_paths = [os.path.join(docs_directory, file) for file in os.listdir(docs_directory) if file.endswith('.txt')]

documents = []

for file_path in file_paths:
    page_content = extract_text_from_txt(file_path)
    document = Document(
        page_content=page_content,
        metadata={"source": "medical_record"}
    )
    documents.append(document)

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

**Create retriever from vector database for RAG chain**

In [ ]:
milvus_retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
# milvus_retriever.invoke("Avulsion", k=1, filter={"source": "medical_record"})

**Define the large language model**

In [ ]:
llm = Ollama(
        base_url="http://llm.ic-shared-llm.svc.cluster.local:11434",
        model="mistral",
        top_p=0.92,
        temperature=0.01,
        num_predict=512,
        repeat_penalty=1.03,
        callbacks=[StreamingStdOutCallbackHandler()]
    )

**Perform Retrievel Augmented Generation**

In [ ]:
retriever = milvus_retriever

system_prompt = (
    "Use the given context to give advice about follow-up actions for the incident. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

input_text = f"fracture is classified as: {record['fracture_classification']} and is described as follows: {record['description']}"

advice = chain.invoke({"input": input_text})

**Update database with result from the LLM**

In [ ]:
update_data = (
    advice,
    incident_id
)

try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()

    cursor.execute(update_record_query, update_data)

    conn.commit()
    print(f"Record with ID {incident_id} updated successfully.")

except psycopg2.DatabaseError as e:
    print(f"Error: {e}")
    conn.rollback()
finally:
    if conn:
        cursor.close()
        conn.close()
        print("Database connection closed.")